# Amateur Hour - Predicting Stocks using LightGBM (Market Data Only)
### Starter Kernel by ``Magichanics`` 
*([GitHub](https://github.com/Magichanics) - [Kaggle](https://www.kaggle.com/magichanics))*

This is more of an improvement in organization and efficiency compared to my previous notebook. The reason why I've decided to create a Market Data Only kernel is because of how small the runtime is.

Feel free to post suggestions or criticisms! 

## Table of Contents

* [Step 1. Cleaning Dataset](#section1)
* [Step 2. Feature Engineering](#section2)
* [Step 3. Modelling using LightGBM](#section3)
* [Step 4. Applying the Model](#section4)

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
from itertools import chain

import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# import environment for data
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()
sampling = False
if sampling:
    market_train_df = market_train_df.tail(400_000)
del news_train_df

<a id='section1'></a>
## Step 1. Cleaning Dataset

We'll be getting rid of a bit of data from the market dataset.

In [ ]:
market_train_df.head()

### Getting rid of Data prior to 2010
Data affected by the Financial Crisis (and January 2010 data) may not benefit this model.

In [ ]:
from datetime import datetime, timedelta
start = datetime(2010, 2, 1, 0, 0, 0).date()
market_train_df = market_train_df.loc[market_train_df['time'].dt.date >= start].reset_index(drop=True)

### Cleaning Data
We will be only keeping the features with good correlation with the dataset.

In [ ]:
def clean_data(market_df, train=True):
    
    # get only what's necessary
    valid_market_cols = ['time', 'assetCode', 'volume', 'close', 'open',\
                           'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',\
                           'returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
    if train:
        valid_market_cols += ['returnsOpenNextMktres10']
    market_df = market_df[valid_market_cols]

    return market_df

In [ ]:
X_train = clean_data(market_train_df, train=True)

In [ ]:
X_train.shape

In [ ]:
del market_train_df

<a id='section2'></a>
## Step 2. Feature Engineering

From Quant features to text processing features.

In [ ]:
X_train.tail()

### Entire Market and Individual Asset Lag Features
We are going to be obtaining Quant Features from both the entire market dataframe and from each individual asset based on assetCode.

Source: https://www.kaggle.com/qqgeogor/eda-script-67

In [ ]:
from multiprocessing import Pool

def create_lag(df_code,n_lag=[3,7,14,],shift_size=1):
    code = df_code['assetCode'].unique()
    
    for col in return_features:
        for window in n_lag:
            rolled = df_code[col].shift(shift_size).rolling(window=window)
            lag_mean = rolled.mean()
            lag_max = rolled.max()
            lag_min = rolled.min()
            lag_std = rolled.std()
            df_code['%s_lag_%s_mean'%(col,window)] = lag_mean
            df_code['%s_lag_%s_max'%(col,window)] = lag_max
            df_code['%s_lag_%s_min'%(col,window)] = lag_min

    return df_code.fillna(-1)

def generate_lag_features(df,n_lag = [3,7,14]):
    
    assetCodes = df['assetCode'].unique()
    all_df = []
    df_codes = df.groupby('assetCode')
    df_codes = [df_code[1][['time','assetCode']+return_features] for df_code in df_codes]
    
    pool = Pool(4)
    all_df = pool.map(create_lag, df_codes)
    
    new_df = pd.concat(all_df)  
    new_df.drop(return_features+['time', 'assetCode'],axis=1,inplace=True)
    new_df = pd.concat([df, new_df], axis=1, sort=False)
    pool.close()
    
    return new_df

In [ ]:
%%time
return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close', 'volume']
X_train = generate_lag_features(X_train)

In [ ]:
X_train.dropna().head()

### Clustering
We will be clustering the open and close features using KMeans.

In [ ]:
from sklearn.cluster import KMeans

# suggesting -> add multiprocessing to KMeans

def clustering(X):

    def cluster_modelling(features):
        df_set = X[features].fillna(0)
        cluster_model = KMeans(n_clusters = 4)
        cluster_model.fit(df_set)
        return cluster_model.predict(df_set)
    
    X['cluster_open_close'] = cluster_modelling(['open', 'close'])
    X['cluster_prev10'] = cluster_modelling(['returnsClosePrevMktres10', 'returnsClosePrevRaw10'])
    
    return X

In [ ]:
%%time
X_train = clustering(X_train)

### Misc. Features
Inclues the following features:
* Daily Difference

In [ ]:
def misc_features(X):
    
    # Adding daily difference
    new_col = X["close"] - X["open"]
    X.insert(loc=6, column="daily_diff", value=new_col)
    X['close_to_open'] =  np.abs(X['close'] / X['open'])
    
    # extra features
    X['bartrend'] = X['close'] / X['open']
    X['average'] = (X['close'] + X['open'])/2
    X['pricevolume'] = X['volume'] * X['close']


In [ ]:
misc_features(X_train)

In [ ]:
X_train.dropna().tail()

<a id='section3'></a>
## Step 3. Modelling using LightGBM

### Preparing Datasets for Modelling

In [ ]:
y_train = X_train['returnsOpenNextMktres10']
X_train = X_train[[f for f in X_train.columns if f not in ['time', 'assetCode', 'universe', 'assetName', 'returnsOpenNextMktres10',
                                                          'headline', 'subjects', 'audiences']]].fillna(0)

### Fixed Training Split
The reason why we need to do a fixed training test split that fetches the last few rows of the training dataset is to avoid odd results, since randomly choosing rows will cause the validation dataset to be filled with rows with different timestamps.

In [ ]:
def fixed_train_test_split(X, y, train_size):
    
    # round train size
    train_size = int(train_size * len(X))
    
    # split data
    X_train, y_train = X[train_size:], y[train_size:]
    X_valid, y_valid = X[:train_size], y[:train_size]
    
    return X_train, y_train, X_valid, y_valid

In [ ]:
X_train, y_train, X_valid, y_valid = fixed_train_test_split(X_train, y_train, 0.85)

### Using LightGBM for modelling

Model from:
https://www.kaggle.com/rabaman/0-64-in-100-lines/code

In [ ]:
import lightgbm as lgb

params = {"objective" : "binary",
          "metric" : "binary_logloss",
          "num_leaves" : 60,
          "max_depth": -1,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1 }

lgtrain, lgval = lgb.Dataset(X_train, y_train), lgb.Dataset(X_valid, y_valid)
lgb_model = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=200)


<a id='section4'></a>
## Step 4. Applying the Model


Predictions will be made through a for loop, and apply all the functions above onto the test dataset.

In [ ]:
def get_X(market_test_df):
    
    X_test = clean_data(market_test_df, train=False)
    X_test = generate_lag_features(X_test)
    X_test = clustering(X_test)
    misc_features(X_test)
    X_test = X_test[[f for f in X_test.columns if f not in ['time', 'assetCode', 'universe', \
                                                            'assetName', 'returnsOpenNextMktres10',\
                                                          'headline', 'subjects', 'audiences']]].fillna(0)
    
    return X_test

def make_predictions(market_obs_df):
    
    # predict using given model
    X_test = get_X(market_obs_df)
    prediction_values = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)  * 2 - 1

    return prediction_values


In [ ]:
%%time
n_days = 0
for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days(): # Looping over days from start of 2017 to 2019-07-15
    
    n_days +=1
    if n_days % 50 == 0:
        print(n_days)
    
    # make predictions
    predictions_template_df['confidenceValue'] = make_predictions(market_obs_df)
    
    # save predictions
    env.predict(predictions_template_df)
    
env.write_submission_file()

**Sources:**
* [Amateur Hour - Using Headlines to Predict Stocks by Magichanics](https://www.kaggle.com/magichanics/amateur-hour-using-headlines-to-predict-stocks)
* [Simple Quant Features by Youhanlee](https://www.kaggle.com/youhanlee/simple-quant-features-using-python)
* [>0.64 in 100 lines by rabaman](https://www.kaggle.com/rabaman/0-64-in-100-lines/comments)
* [eda script 67 by qqgeogor](https://www.kaggle.com/qqgeogor/eda-script-67)